## Set Up of the Neo4j database

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
from pathlib import Path
import pickle

# Get the absolute path to the project root (parent of notebooks directory)
# This assumes the notebook is running in the 'notebooks' folder
PROJECT_ROOT = Path.cwd().parent

# Add project root to sys.path if not present
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# Add src directory to sys.path
SRC_DIR = PROJECT_ROOT / "src"
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

# Path to the pickle file relative to current working directory
PKL_PATH = Path("./funds_backup_metadata.pkl")

print("Current working directory:", Path.cwd())
print("Project Root:", PROJECT_ROOT)
print("Source Directory:", SRC_DIR)
print("PKL_PATH resolves to:", PKL_PATH.resolve())

# Load the pickle file
with PKL_PATH.open("rb") as f:
    funds_total = pickle.load(f)

print(f"Loaded {len(funds_total)} funds from pickle file")

Current working directory: /home/luis/Desktop/code/RAG/notebooks
Project Root: /home/luis/Desktop/code/RAG
Source Directory: /home/luis/Desktop/code/RAG/src
PKL_PATH resolves to: /home/luis/Desktop/code/RAG/notebooks/funds_backup.pkl
Loaded 420 funds from pickle file


Set up the Neo4j instance using docker-compose

In [4]:
PROJECT_ROOT = Path.cwd().parent

# Add project root to sys.path if not present
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# Add src directory to sys.path
SRC_DIR = PROJECT_ROOT / "src"
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))
%reload_ext autoreload

from simple_rag.database.neo4j.neo4j import Neo4jDatabase

neo = Neo4jDatabase(auto_start=True)
# Reset the entire database
neo.reset_database()
neo.create_indexes()
neo.create_constraints()


🗑️  Dropping all constraints...
   ✅ Dropped 12 constraints
🗑️  Dropping all indexes...
   ✅ Dropped 9 indexes
🗑️  Deleting 0 nodes from database...
✅ Database reset complete - all data, constraints, and indexes removed
⚙️  Initializing Graph Schema & Indexes...
   ... Applying Vector Indexes
   ... Applying Unique Constraints
   ... Applying Fulltext Indexes

🔎 VERIFYING INDEX STATUS:
INDEX NAME                     | TYPE            | STATE      | PROVIDER
---------------------------------------------------------------------------
commentary_vector_index        | VECTOR          | ONLINE     | vector-3.0
fundNameIndex                  | FULLTEXT        | ONLINE     | fulltext-2.0
fund_name_index                | RANGE           | ONLINE     | range-1.0
fund_ticker_unique             | RANGE           | ONLINE     | range-1.0
holding_id_unique              | RANGE           | ONLINE     | range-1.0
objective_vector_index         | VECTOR          | ONLINE     | vector-3.0
perf_id_uniqu

### Create the Providers

In [5]:
provider = ['The Vanguard Group, Inc', 'BlackRock, Inc']

for p in provider:
    neo.get_or_create_provider(p)

In [6]:
registrants = set()
for fund in funds_total:
    if '\xa0' in fund.registrant:
        fund.registrant = fund.registrant.replace('\xa0', ' ')
    registrants.add(fund.registrant)
print(registrants)
# Initialize the dictionary with lists
trusts = {
    'The Vanguard Group, Inc': [],
    'BlackRock, Inc': []
}

# Process registrants
for registrant in registrants:
    if 'Vanguard' in registrant:
        trusts['The Vanguard Group, Inc'].append(registrant) 
    else:
        trusts['BlackRock, Inc'].append(registrant)
print(trusts)

{'Vanguard Specialized Funds', 'Vanguard Whitehall Funds', 'Vanguard Index Funds', 'Vanguard World Fund', 'iShares Trust'}
{'The Vanguard Group, Inc': ['Vanguard Specialized Funds', 'Vanguard Whitehall Funds', 'Vanguard Index Funds', 'Vanguard World Fund'], 'BlackRock, Inc': ['iShares Trust']}


### Create the Trusts

In [7]:

for key, value in trusts.items():
    for provider in value:
        print("Created: ", key, " ->", provider)
        neo.get_or_create_trust(key, [provider])

Created:  The Vanguard Group, Inc  -> Vanguard Specialized Funds
Created:  The Vanguard Group, Inc  -> Vanguard Whitehall Funds
Created:  The Vanguard Group, Inc  -> Vanguard Index Funds
Created:  The Vanguard Group, Inc  -> Vanguard World Fund
Created:  BlackRock, Inc  -> iShares Trust


### Create the Share classes

In [8]:
share_classes_data = [
    {
        "name": "Admiral Shares",
        "clean_name": "Admiral Shares",
        "description": (
            "Admiral Shares are Vanguard’s main retail mutual fund share class and typically "
            "have lower expense ratios than legacy Investor Shares. They are intended for "
            "long-term individual investors and usually require a few thousand dollars as a "
            "minimum investment (often around $3,000, though it varies by fund)."
        ),
    },
    {
        "name": "Investor Shares",
        "clean_name": "Investor Shares",
        "description": (
            "Investor Shares are Vanguard’s legacy entry-level mutual fund share class. They "
            "generally have higher expense ratios than Admiral Shares and historically had "
            "lower minimum investments (often around $1,000–$3,000, depending on the fund). "
            "In many cases they are closed to new investors and may be automatically converted "
            "to Admiral Shares once the account balance meets the Admiral minimum."
        ),
    },
    {
        "name": "ETF Shares",
        "clean_name": "ETF Shares",
        "description": (
            "ETF (Exchange-Traded Fund) Shares trade on stock exchanges throughout the day like "
            "a stock. They generally provide the same portfolio exposure as a corresponding "
            "mutual fund share class but offer intraday liquidity. The minimum investment is "
            "the market price of one share (often tens to a few hundred dollars). ETFs are "
            "often more tax-efficient than mutual funds due to the in-kind creation/redemption "
            "mechanism, though investors may face bid–ask spreads."
        ),
    },
    {
        "name": "Institutional Shares",
        "clean_name": "Institutional Shares",
        "description": (
            "Institutional Shares are designed for large investors such as retirement plans, "
            "advisors, endowments, and other institutions. They typically have very low expense "
            "ratios and usually require large minimum investments, commonly in the millions of "
            "dollars (the exact amount varies by fund). Most individuals cannot buy them directly "
            "unless they access them through an employer plan or an institutional platform."
        ),
    },
    {
        "name": "Institutional Plus Shares",
        "clean_name": "Institutional Plus Shares",
        "description": (
            "Institutional Plus Shares are a higher tier of institutional pricing with even lower "
            "expense ratios, generally available only to very large investors. Minimums are typically "
            "in the tens to hundreds of millions of dollars, depending on the fund and access channel."
        ),
    },
    {
        "name": "Institutional Select Shares",
        "clean_name": "Institutional Select Shares",
        "description": (
            "Institutional Select Shares are among the lowest-cost share classes and are typically "
            "available only through very large institutional relationships (often aggregate or negotiated "
            "minimums, potentially in the hundreds of millions to billions). Pricing and availability are "
            "often bespoke and not consistently offered or publicly listed across all funds."
        ),
    },
]


In [9]:
description_map = {sc["clean_name"]: sc["description"] for sc in share_classes_data}
# Cell 2: Count and create share classes with descriptions
from src.simple_rag.models.fund import ShareClassType
from collections import Counter
# Count funds by share class
share_counts = Counter()
for fund in funds_total:
    if fund.share_class:
        if fund.share_class == ShareClassType.OTHER:
            fund.share_class = ShareClassType.ETF
        share_counts[fund.share_class] += 1

for share_type, count in share_counts.items():
    share_class_name = share_type.value
    
    # Get description from mapping, or use a default
    description = description_map.get(share_class_name, f"Share class for {share_class_name}")
    
    # Create share class with proper description
    neo.get_or_create_share_class(share_class_name, description)
    
    print(f"  {share_class_name}: {count} funds")

  Investor Shares: 27 funds
  ETF Shares: 330 funds
  Admiral Shares: 40 funds
  Institutional Shares: 16 funds
  Institutional Plus Shares: 4 funds
  Institutional Select Shares: 3 funds


### Chunking and Embedding of the Strategy

In [9]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Document
from tqdm import tqdm
from src.simple_rag.models.fund import ContentChunk # Assuming you saved the class here
from simple_rag.embeddings.embedding import EmbedData
from simple_rag.models.fund import FundData

from simple_rag.embeddings.embedding import HuggingFaceEmbedding
import re
# --- CONFIGURATION ---
# Larger than Risks (256) because Strategy needs more narrative context
STRAT_CHUNK_SIZE = 512 
STRAT_OVERLAP = 100
BATCH_SIZE = 32

print("⏳ Initializing Strategy Splitter...")
strategy_splitter = SentenceSplitter(
    chunk_size=STRAT_CHUNK_SIZE, 
    chunk_overlap=STRAT_OVERLAP
)

embedder = EmbedData(
    model_name="nomic-ai/nomic-embed-text-v1.5", 
    batch_size=BATCH_SIZE
)

# Global queue for embedding (Pass 2)
strategy_tasks = []

print("🧠 Processing Fund Strategies...")

for fund in tqdm(funds_total, desc="Chunking Strategies"):
    # Skip empty
    if not fund.strategies or fund.strategies in ["N/A", "None", ""]:
        continue
        
    # 1. Create Document
    doc = Document(text=fund.strategies)
    
    # 2. Split
    # The SentenceSplitter is smart:
    # - If text is 91 tokens -> Returns 1 node (Whole text).
    # - If text is 3409 tokens -> Returns ~7 nodes.
    nodes = strategy_splitter.get_nodes_from_documents([doc])
    
    # Reset list
    fund.strategies_chunks = []
    
    for i, node in enumerate(nodes):
        # 3. Create Title
        # Since strategies lack clear headers, we use a sequential naming convention
        # unless we can find a bold header at the start.
        title = f"Investment Strategy - Part {i+1}"
        
        # Create Chunk Object
        new_chunk = ContentChunk(
            id=f"{fund.ticker}_strat_{i}",
            title=title,
            text=node.text,
            embedding=[] # Placeholder
        )
        
        # Add to Fund and Task Queue
        fund.strategies_chunks.append(new_chunk)
        strategy_tasks.append((fund, new_chunk))

# ==================================================
# BATCH EMBEDDING (Reuse your embedder)
# ==================================================
if strategy_tasks:
    print(f"🔄 Embedding {len(strategy_tasks)} strategy chunks...")
    
    # Extract text
    texts = [task[1].text for task in strategy_tasks]
    
    # Embed
    vectors = embedder.embed(texts)
    
    # Assign back
    for i, vector in enumerate(vectors):
        strategy_tasks[i][1].embedding = vector
        
    print(f"✅ Strategy processing complete. Generated {len(strategy_tasks)} nodes.")
else:
    print("⚠️ No strategies found.")

/home/alvar/anaconda3/envs/rag-env/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'validate_default' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'validate_default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(


⏳ Initializing Strategy Splitter...


<All keys matched successfully>


🧠 Processing Fund Strategies...


Chunking Strategies: 100%|██████████| 420/420 [00:00<00:00, 763.43it/s]


🔄 Embedding 421 strategy chunks...


Embedding: 100%|██████████| 421/421 [00:09<00:00, 46.68doc/s]

✅ Strategy processing complete. Generated 421 nodes.


### Chunking and Embedding of the Risks

In [10]:

from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Document
from tqdm import tqdm
from simple_rag.embeddings.embedding import EmbedData
from simple_rag.models.fund import FundData
from simple_rag.models.fund import ContentChunk
from simple_rag.embeddings.embedding import HuggingFaceEmbedding
import re
from typing import List, Tuple

embed_model = HuggingFaceEmbedding(
    model_name="nomic-ai/nomic-embed-text-v1.5", 
    trust_remote_code=True
)

BATCH_SIZE = 32
CHUNK_SIZE = 256  
CHUNK_OVERLAP = 50

print("⏳ Initializing models...")
embedder = EmbedData(
    model_name="nomic-ai/nomic-embed-text-v1.5", 
    batch_size=BATCH_SIZE
)

splitter = SentenceSplitter(
    chunk_size=CHUNK_SIZE, 
    chunk_overlap=CHUNK_OVERLAP
)

# 2. HELPER: Generate Title from Chunk
def generate_title(text: str, index: int) -> str:
    """Extracts a short title from the chunk text."""
    # Try to grab the first sentence or header up to 50 chars
    first_line = text.split('\n')[0].strip()
    # If it starts with "Credit Risk.", grab just that
    if "." in first_line[:50]:
        return first_line.split(".")[0]
    return f"Risk Section {index + 1}"

def clean_noise(text):
    """
    Surgically removes 'S-' followed by digits and merges the 
    surrounding text, leaving the rest of the document formatting untouched.
    """
    pattern = r'\s*S-\d+\s*'
    
    # Replace the whole match with a SINGLE space
    return re.sub(pattern, ' ', text)


embedding_tasks: List[Tuple[FundData, ContentChunk]] = []
for fund in tqdm(funds_total, desc="Chunking Funds"):
    # Skip if no risks
    if not fund.risks or fund.risks in ["N/A", "None", ""]:
        continue
    
    fund.risks = clean_noise(fund.risks)
    # Reset existing chunks if any
    fund.risks_chunks = []
    
    # Create LlamaIndex Document
    doc = Document(text=fund.risks)
    
    # Run Splitter
    nodes = splitter.get_nodes_from_documents([doc])
    
    # Convert Nodes to ContentChunk objects
    for i, node in enumerate(nodes):
        # Create the Chunk Object
        new_chunk = ContentChunk(
            id=f"{fund.ticker}_risk_{i}",
            title=generate_title(node.text, i),
            text=node.text,
            embedding=[] # Empty for now
        )
        
        # Add to the global queue for embedding
        embedding_tasks.append((fund, new_chunk))

if embedding_tasks:
    # Extract just the text strings for the embedder
    texts_to_embed = [task[1].text for task in embedding_tasks]

    # Run the batch embedding using your EmbedData class
    # This efficiently processes texts in batches of 32 on the GPU
    vectors = embedder.embed(texts_to_embed)

    # ==================================================
    # PASS 3: ASSIGN & LINK
    # ==================================================
    print("🔗 Linking vectors back to funds...")

    for i, vector in enumerate(vectors):
        # Retrieve the original objects
        fund_obj, chunk_obj = embedding_tasks[i]
        
        # 1. Assign vector to the chunk
        chunk_obj.embedding = vector
        
        # 2. Add the completed chunk to the Fund's list
        fund_obj.risks_chunks.append(chunk_obj)

    print(f"✅ DONE! Processed {len(funds_total)} funds.")
    
    # --- VERIFICATION ---
    sample_fund = next((f for f in funds_total if f.risks_chunks), None)
    if sample_fund:
        print(f"\nExample Fund: {sample_fund.ticker}")
        print(f"Generated {len(sample_fund.risks_chunks)} Risk Nodes.")
        print(f"First Node Title: '{sample_fund.risks_chunks[0].title}'")
        print(f"Vector Length: {len(sample_fund.risks_chunks[0].embedding)}")
else:
    print("⚠️ No risk chunks found to embed.")


<All keys matched successfully>


⏳ Initializing models...


<All keys matched successfully>
Embedding: 100%|██████████| 2422/2422 [00:26<00:00, 91.69doc/s] 

🔗 Linking vectors back to funds...
✅ DONE! Processed 420 funds.

Example Fund: VEXMX
Generated 3 Risk Nodes.
First Node Title: 'Risk Section 1'
Vector Length: 768


### Performance Commentary and Objective

In [11]:
from typing import List, Tuple
from tqdm import tqdm


print("⏳ Starting Single-Field Embeddings (Objective & Performance)...")

# =========================================================
# 1. COLLECT TEXTS (Prepare Batches)
# =========================================================
# We use lists to store the (Fund, Text) tuples so we can map vectors back later
objective_tasks: List[Tuple[FundData, str]] = []
performance_tasks: List[Tuple[FundData, str]] = []

print("📋 Scanning funds for valid text...")

for fund in funds_total:
    # --- A. OBJECTIVE ---
    # specific check to ignore empty or "N/A" placeholders
    if fund.objective and str(fund.objective).strip() not in ["N/A", "None", ""]:
        objective_tasks.append((fund, fund.objective))

    # --- B. PERFORMANCE COMMENTARY ---
    if fund.performance_commentary and str(fund.performance_commentary).strip() not in ["N/A", "None", ""]:
        performance_tasks.append((fund, fund.performance_commentary))

# =========================================================
# 2. EMBED OBJECTIVES
# =========================================================
if objective_tasks:
    print(f"🔄 Embedding {len(objective_tasks)} objectives...")
    
    # 1. Extract just the text strings for the embedder
    obj_texts = [task[1] for task in objective_tasks]
    
    # 2. Run Batch Embedding (Uses GPU efficiently)
    obj_vectors = embedder.embed(obj_texts)
    
    # 3. Assign vectors back to the specific Fund objects
    for i, vector in enumerate(obj_vectors):
        fund_obj = objective_tasks[i][0]
        # Direct assignment to the list[float] field
        fund_obj.objective_embedding = vector 
else:
    print("⚠️ No valid objectives found.")

# =========================================================
# 3. EMBED PERFORMANCE COMMENTARIES
# =========================================================
if performance_tasks:
    print(f"🔄 Embedding {len(performance_tasks)} performance commentaries...")
    
    # 1. Extract text strings
    perf_texts = [task[1] for task in performance_tasks]
    
    # 2. Run Batch Embedding
    perf_vectors = embedder.embed(perf_texts)
    
    # 3. Assign vectors back to Fund objects
    for i, vector in enumerate(perf_vectors):
        fund_obj = performance_tasks[i][0]
        # Direct assignment to the list[float] field
        fund_obj.performance_commentary_embedding = vector
else:
    print("⚠️ No valid performance commentaries found.")

print(f"✅ Single-field processing complete.")

# =========================================================
# 4. VERIFICATION
# =========================================================
# Let's inspect the first fund that actually had data to ensure it worked
sample_fund = next((f for f in funds_total if f.objective_embedding), None)

if sample_fund:
    print(f"\n🔍 INSPECTION: Fund '{sample_fund.ticker}'")
    
    if sample_fund.objective_embedding:
        print(f"   ✅ Objective Vector Size: {len(sample_fund.objective_embedding)}")
    else:
        print("   ❌ Objective Vector: Missing")

    if sample_fund.performance_commentary_embedding:
        print(f"   ✅ Performance Vector Size: {len(sample_fund.performance_commentary_embedding)}")
    else:
        print("   ❌ Performance Vector: Missing")

⏳ Starting Single-Field Embeddings (Objective & Performance)...
📋 Scanning funds for valid text...
🔄 Embedding 188 objectives...


Embedding: 100%|██████████| 188/188 [00:00<00:00, 201.59doc/s]


🔄 Embedding 383 performance commentaries...


Embedding: 100%|██████████| 383/383 [00:05<00:00, 70.69doc/s]

✅ Single-field processing complete.

🔍 INSPECTION: Fund 'VEXMX'
   ✅ Objective Vector Size: 768
   ✅ Performance Vector Size: 768


In [12]:
import pickle
from pathlib import Path
import sys

# Add RAG directory to path
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))

# Define pickle file path
PKL_PATH = Path("./funds_backup.pkl")

print("Current working directory:", Path.cwd())
print("PKL_PATH resolves to:", PKL_PATH.resolve())

# Save to pickle file
try:
    with PKL_PATH.open("wb") as f:
        pickle.dump(funds_total, f)
    
    print(f"Successfully saved {len(funds_total)} funds to pickle file")
    print(f"File size: {PKL_PATH.stat().st_size / 1024:.2f} KB")
    
except Exception as e:
    print(f"Error saving to pickle file: {e}")

Current working directory: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks
PKL_PATH resolves to: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/funds_backup.pkl
Successfully saved 420 funds to pickle file
File size: 76535.78 KB


### Create the funds

In [10]:

providers = set()
for fund in funds_total:
    fund.provider = fund.provider.rstrip('.')
    providers.add(fund.provider)

providers

{'BlackRock, Inc', 'The Vanguard Group, Inc'}

In [11]:
for fund in funds_total:
    neo.create_fund(fund)

📊 Creating Profile 2024-12-31 for: VEXMX


/home/luis/Desktop/code/RAG/.venv/lib/python3.11/site-packages/neo4j/_sync/work/result.py:620: UserWarning: Expected a result with a single record, but found multiple.
  warn(


📊 Creating Profile 2024-12-31 for: VXF
📊 Creating Profile 2024-12-31 for: VEXAX
📊 Creating Profile 2024-12-31 for: VIEIX
📊 Creating Profile 2024-12-31 for: VEMPX
📊 Creating Profile 2024-12-31 for: VSEMX
📊 Creating Profile 2024-12-31 for: VIMSX
📊 Creating Profile 2024-12-31 for: VO
📊 Creating Profile 2024-12-31 for: VIMAX
📊 Creating Profile 2024-12-31 for: VMCIX
📊 Creating Profile 2024-12-31 for: VMCPX
📊 Creating Profile 2024-12-31 for: VMGIX
📊 Creating Profile 2024-12-31 for: VOT
📊 Creating Profile 2024-12-31 for: VMGMX
📊 Creating Profile 2024-12-31 for: VMVIX
📊 Creating Profile 2024-12-31 for: VOE
📊 Creating Profile 2024-12-31 for: VMVAX
📊 Creating Profile 2024-12-31 for: NAESX
📊 Creating Profile 2024-12-31 for: VB
📊 Creating Profile 2024-12-31 for: VSMAX
📊 Creating Profile 2024-12-31 for: VSCIX
📊 Creating Profile 2024-12-31 for: VSCPX
📊 Creating Profile 2024-12-31 for: VISGX
📊 Creating Profile 2024-12-31 for: VBK
📊 Creating Profile 2024-12-31 for: VSGAX
📊 Creating Profile 2024-12-31 

### Insert the Charts

In [12]:
import os
import re
from pathlib import Path
from typing import List, Dict, Optional
from tqdm import tqdm
from src.simple_rag.models.fund import ChartData

def extract_ticker_regex(filename: str) -> Optional[str]:
    """Extract ticker using regex patterns"""
    name = Path(filename).stem
    
    # Common ticker patterns
    patterns = [
        r'^([a-zA-Z]{1,5})[_-]',        # VTI_performance.svg or vti_performance.svg
        r'[_-]([a-zA-Z]{1,5})$',        # performance_VTI.svg or performance_vti.svg
        r'^([a-zA-Z]{1,5})(?=[a-z])',   # VTIperformance.svg or vti_performance.svg
    ]
    
    for pattern in patterns:
        match = re.search(pattern, name)
        if match:
            return match.group(1).upper()
    return None

def process_svgs_simple(folder_path: Path, title: str) -> Dict[str, List[ChartData]]:
    """Simple SVG processing approach"""
    charts_by_ticker = {}
    svg_files = list(folder_path.glob("*.svg"))
    
    for svg_file in tqdm(svg_files, desc="Processing SVG files"):
        ticker = extract_ticker_regex(svg_file.name)
        if not ticker:
            print(f"No ticker found in {svg_file.name}")
            continue
            
        with open(svg_file, 'r') as f:
            svg_content = f.read()
        chart = ChartData(
            title=title + ": " + ticker,
            image_type=title,  # Default type
            svg_content=svg_content
        )
        charts_by_ticker.setdefault(ticker, []).append(chart)
    
    return charts_by_ticker

portfolio_path = Path("/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/charts/portfolio")
print(portfolio_path)
portfolio_charts = process_svgs_simple(portfolio_path, "Portfolio")
performance_path = Path("/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/charts/performance")
print(performance_path)
performance_charts = process_svgs_simple(performance_path, "Performance")
print(performance_charts)
holdings_path = Path("/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/charts/holdings")
print(holdings_path)
holdings_charts = process_svgs_simple(holdings_path, "Holdings")
print(holdings_charts)

total_charts = {**portfolio_charts, **performance_charts, **holdings_charts}
print(total_charts)
print(len(total_charts))

total_charts = {**portfolio_charts, **performance_charts, **holdings_charts}

for ticker, charts in total_charts.items():
    print(f"Processing {len(charts)} charts for {ticker}")
    
    for chart in charts:
        neo.add_chart_to_fund(
            fund_ticker=ticker,
            title=chart.title,
            category=chart.image_type,  # or chart.category if you have that field
            svg_content=chart.svg_content,
            date="2024-12-31"  
        )
        print(f"Adding chart: {chart.title} to fund {ticker}")
        neo.add_chart_to_fund(
            fund_ticker=ticker,
            title=chart.title,
            category=chart.image_type,  # or chart.category if you have that field
            svg_content=chart.svg_content,
            date="2024-12-31"  
        )


/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/charts/portfolio


Processing SVG files: 0it [00:00, ?it/s]


/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/charts/performance


Processing SVG files: 0it [00:00, ?it/s]


{}
/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/charts/holdings


Processing SVG files: 0it [00:00, ?it/s]

{}
{}
0


### Add the assets

In [13]:
count = 0 
for fund in funds_total:
    if fund.non_derivatives is None:
        print(fund.name)
        count += 1
print(count)

Vanguard Extended Market Index Fund
Vanguard Extended Market Index Fund
Vanguard Extended Market Index Fund
Vanguard Extended Market Index Fund
Vanguard Extended Market Index Fund
Vanguard Mid-Cap Index Fund
Vanguard Mid-Cap Index Fund
Vanguard Mid-Cap Index Fund
Vanguard Mid-Cap Index Fund
Vanguard Mid-Cap Index Fund
Vanguard Mid-Cap Growth Index Fund
Vanguard Mid-Cap Growth Index Fund
Vanguard Mid-Cap Value Index Fund
Vanguard Mid-Cap Value Index Fund
Vanguard Small-Cap Index Fund
Vanguard Small-Cap Index Fund
Vanguard Small-Cap Index Fund
Vanguard Small-Cap Index Fund
Vanguard Small-Cap Growth Index Fund
Vanguard Small-Cap Growth Index Fund
Vanguard Small-Cap Growth Index Fund
Vanguard Small-Cap Value Index Fund
Vanguard Small-Cap Value Index Fund
Vanguard Small-Cap Value Index Fund
Vanguard Total Stock Market Index Fund
Vanguard Total Stock Market Index Fund
Vanguard Total Stock Market Index Fund
Vanguard Total Stock Market Index Fund
Vanguard Total Stock Market Index Fund
Vanguard

In [14]:
count = 0
series = {}
for fund in funds_total:
    if fund.non_derivatives is not None:
        series[fund.series_id] = fund.non_derivatives

print(count)

0


In [15]:
from tqdm import tqdm
import time
for fund in tqdm(funds_total, desc="Processing funds"):
    
    if not hasattr(fund, 'series_id') or fund.series_id is None:
        continue
    
    series_id = fund.series_id  
    holding = series[series_id].holdings_df
    report_date = series[series_id].date

    t0 = time.perf_counter()
    neo.create_fund_holdings(fund.ticker, series_id, report_date, holding)
    t1 = time.perf_counter()
    print(f"Processed {fund.ticker} in {t1-t0:.2f} seconds")

Processing funds:   0%|          | 0/420 [00:00<?, ?it/s]

💰 Processing 3458 holdings for S000002841...
   ⏳ VEXMX: Ingested batch 0 to 1000...
   ⏳ VEXMX: Ingested batch 1000 to 2000...
   ⏳ VEXMX: Ingested batch 2000 to 3000...


Processing funds:   0%|          | 1/420 [00:01<07:05,  1.02s/it]

   ⏳ VEXMX: Ingested batch 3000 to 3458...
✅ Successfully ingested 3458 holdings for VEXMX
Processed VEXMX in 1.02 seconds
💰 Processing 3458 holdings for S000002841...
   ⏳ VXF: Ingested batch 0 to 1000...


Processing funds:   0%|          | 2/420 [00:01<04:16,  1.63it/s]

...

✅ Successfully ingested 714 holdings for IBDV
Processed IBDV in 0.07 seconds
💰 Processing 1206 holdings for S000085095...
   ⏳ IBMS: Ingested batch 0 to 1000...
   ⏳ IBMS: Ingested batch 1000 to 1206...
✅ Successfully ingested 1206 holdings for IBMS
Processed IBMS in 0.13 seconds
💰 Processing 462 holdings for S000072204...
   ⏳ IBDW: Ingested batch 0 to 462...
✅ Successfully ingested 462 holdings for IBDW
Processed IBDW in 0.05 seconds
💰 Processing 375 holdings for S000085096...


Processing funds:  97%|█████████▋| 409/420 [00:31<00:00, 17.66it/s]

   ⏳ IBDZ: Ingested batch 0 to 375...
✅ Successfully ingested 375 holdings for IBDZ
Processed IBDZ in 0.04 seconds
💰 Processing 5 holdings for S000085229...
   ⏳ IBTP: Ingested batch 0 to 5...
✅ Successfully ingested 5 holdings for IBTP
Processed IBTP in 0.01 seconds
💰 Processing 9 holdings for S000085230...
   ⏳ IBGA: Ingested batch 0 to 9...
✅ Successfully ingested 9 holdings for IBGA
Processed IBGA in 0.01 seconds
💰 Processing 4 holdings for S000085232...
   ⏳ IBGK: Ingested batch 0 to 4...
✅ Successfully ingested 4 holdings for IBGK
Processed IBGK in 0.01 seconds
💰 Processing 2 holdings for S000085097...
   ⏳ IBIK: Ingested batch 0 to 2...
✅ Successfully ingested 2 holdings for IBIK
Processed IBIK in 0.01 seconds
💰 Processing 891 holdings for S000023614...
   ⏳ IGOV: Ingested batch 0 to 891...
✅ Successfully ingested 891 holdings for IGOV
Processed IGOV in 0.08 seconds
💰 Processing 3 holdings for S000076781...
   ⏳ LQDW: Ingested batch 0 to 3...
✅ Successfully ingested 3 holdings f

Processing funds: 100%|██████████| 420/420 [00:31<00:00, 13.18it/s]

   ⏳ EMB: Ingested batch 0 to 662...
✅ Successfully ingested 662 holdings for EMB
Processed EMB in 0.07 seconds
💰 Processing 3 holdings for S000083814...
   ⏳ IWMW: Ingested batch 0 to 3...
✅ Successfully ingested 3 holdings for IWMW
Processed IWMW in 0.01 seconds
💰 Processing 3 holdings for S000083815...
   ⏳ IVVW: Ingested batch 0 to 3...
✅ Successfully ingested 3 holdings for IVVW
Processed IVVW in 0.01 seconds
💰 Processing 635 holdings for S000060088...
   ⏳ SYSB: Ingested batch 0 to 635...
✅ Successfully ingested 635 holdings for SYSB
Processed SYSB in 0.06 seconds
💰 Processing 10 holdings for S000044143...
   ⏳ TFLO: Ingested batch 0 to 10...
✅ Successfully ingested 10 holdings for TFLO
Processed TFLO in 0.01 seconds
💰 Processing 331 holdings for S000063466...
   ⏳ BGRN: Ingested batch 0 to 331...
✅ Successfully ingested 331 holdings for BGRN
Processed BGRN in 0.04 seconds


### Company Filings

In [9]:
from src.simple_rag.utils.cache_manager import warm_cache_with_persistence

tickers = [
    'AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA', 
    'NVDA', 'META', 'NFLX', 'DIS', 'KO', 
    'PEP', 'COST', 'WMT', 'NKE', 'SBUX', 
    'PFE', 'JNJ', 'XOM', 'V', 'PYPL'
]

warm_cache_with_persistence(tickers)


Starting download for 20 companies...
Cache directory: /home/luis/Desktop/code/RAG/notebooks/edgar_cache


Companies:   0%|          | 0/20 [00:00<?, ?it/s]

[AAPL] 10-K downloaded and cached
[AAPL] DEF 14A downloaded and cached


Companies:   5%|▌         | 1/20 [00:08<02:45,  8.70s/it]

[AAPL] Downloaded 20 Form 4s
[MSFT] 10-K downloaded and cached
[MSFT] DEF 14A downloaded and cached


...

[XOM] Downloaded 35 Form 4s
[V] 10-K downloaded and cached
[V] DEF 14A downloaded and cached


Companies:  95%|█████████▌| 19/20 [05:47<00:12, 12.52s/it]

[V] Downloaded 31 Form 4s
[PYPL] 10-K downloaded and cached
[PYPL] DEF 14A downloaded and cached


Companies: 100%|██████████| 20/20 [05:58<00:00, 17.93s/it]

[PYPL] Downloaded 27 Form 4s

CACHE SUMMARY
Files downloaded: 852
Files already cached: 0
Errors: 0
Total files in cache: 852
Cache size: 3.08 MB


In [11]:
from edgar import set_identity, Company, use_local_storage
import re
import json
from src.simple_rag.extraction.company_filings import CompanyExtractor
from src.simple_rag.models.company import CompanyEntity, FilingMetadata
from src.simple_rag.utils.cache_manager import (
    is_cached, 
    load_from_cache, 
    save_to_cache,
    get_cache_path
)

set_identity("luis.alvarez.conde@alumnos.upm.es")
use_local_storage("./edgar_cache")

from IPython.display import display, Markdown
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm

companies = {}


# For the mockup we are going to use the following 20 companies
tickers = [
    'AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA', 
    'NVDA', 'META', 'NFLX', 'DIS', 'KO', 
    'PEP', 'COST', 'WMT', 'NKE', 'SBUX', 
    'PFE', 'JNJ', 'XOM', 'V', 'PYPL'
]

def process_insider_batch(batch_data):
    """
    Process a batch of insider filings - runs in separate process.
    Args: List of (summary, filing_url) tuples
    Returns: List of all transactions from the batch
    """
    try:
        extractor = CompanyExtractor()
        return extractor.extract_insider_transactions_batch(batch_data)
    except Exception as e:
        print(f"Batch error: {e}")
        return []

for ticker in tqdm(tickers, desc="Processing companies"):
    try:
        company = Company(ticker)

        # ==================== 10-K FILING ====================
        tenk_filing = company.get_filings(form="10-K").latest()
        
        # Try to load from cache first
        if is_cached(ticker, "10-K", tenk_filing.accession_number):
            print(f"[{ticker}] Loading 10-K from cache...")
            tenk = load_from_cache(ticker, "10-K", tenk_filing.accession_number)
        else:
            print(f"[{ticker}] Downloading 10-K...")
            tenk = tenk_filing.obj()
            save_to_cache(ticker, "10-K", tenk_filing.accession_number, tenk)

        # ==================== EXECUTIVE COMPENSATION ====================
        exec_comp_filing = company.get_filings(form="DEF 14A").latest()
        
        if exec_comp_filing:
            if is_cached(ticker, "DEF14A", exec_comp_filing.accession_number):
                print(f"[{ticker}] Loading DEF 14A from cache...")
                exec_comp_obj = load_from_cache(ticker, "DEF14A", exec_comp_filing.accession_number)
            else:
                print(f"[{ticker}] Downloading DEF 14A...")
                exec_comp_obj = exec_comp_filing.obj()
                save_to_cache(ticker, "DEF14A", exec_comp_filing.accession_number, exec_comp_obj)
        else:
            exec_comp_obj = None

        # ==================== INSIDER TRADES ====================
        ins_filings = company.get_filings(form="4").filter(date='2025-09-01:')

        # Extract income statement
        income_statement = tenk.income_statement
        
        print(f"[{ticker}] Period of report: {tenk_filing.period_of_report}")
        
        metadata = FilingMetadata(
            accession_number=tenk_filing.accession_no,
            filing_type=tenk_filing.form,
            filing_date=tenk_filing.filing_date,
            report_period_end=tenk_filing.period_of_report,
            filing_url=tenk_filing.url,
            cik=str(tenk_filing.cik)
        )
        
        print(f"[{ticker}] Filing Company: {tenk.company}")
        
        # Create CompanyEntity
        companies[ticker] = CompanyEntity(
            name=tenk.company, 
            cik=str(tenk_filing.cik), 
            ticker=ticker
        )
        
        # Extract and process income statement
        df = income_statement.to_dataframe()
        statement_dict = extractor.extract_income_statement_dict(df)
        extractor.process_income_statement_dict(
            companies[ticker], 
            statement_dict, 
            metadata, 
            str(income_statement)
        )
        
        # Get the Filing10K object that was just created
        filing_10k = companies[ticker].filings_10k[metadata.filing_date]
        
        # Assign the other sections to the Filing10K object
        filing_10k.business_information = tenk["Item 1"]
        filing_10k.risk_factors = tenk["Item 1A"]
        filing_10k.balance_sheet_text = str(tenk.balance_sheet)
        filing_10k.cash_flow_text = str(tenk.cash_flow_statement)
        filing_10k.management_discussion_and_analysis = tenk["Item 7"]
        filing_10k.legal_proceedings = tenk["Item 3"]
        filing_10k.properties = tenk["Item 2"]
        
        # Extract executive compensation
        if exec_comp_obj:
            extractor.extract_executive_compensation(
                companies[ticker], 
                exec_comp_obj, 
                metadata, 
                exec_comp_filing.url
            )
        
        # Extract insider transactions
        print(f"[{ticker}] Found {len(ins_filings)} insider filings")
        
        if len(ins_filings) > 100:
            print(f"[{ticker}] ⚠️ WARNING: {len(ins_filings)} insider transactions - this may take a while")

        insider_args = []
        cached_count = 0
        downloaded_count = 0
        
        for ins_filing in ins_filings:
            try:
                # Try to load the FULL Form 4 object from cache first
                if is_cached(ticker, "Form4", ins_filing.accession_number):
                    ins_obj = load_from_cache(ticker, "Form4", ins_filing.accession_number)
                    cached_count += 1
                else:
                    ins_obj = ins_filing.obj()
                    save_to_cache(ticker, "Form4", ins_filing.accession_number, ins_obj)
                    downloaded_count += 1
                
                # NOW get the summary from the (cached or fresh) object
                summary = ins_obj.get_ownership_summary()
                insider_args.append((summary, ins_filing.url))
                
            except Exception as e:
                print(f"[{ticker}] Error processing insider filing: {e}")
        if cached_count > 0:
            print(f"[{ticker}] Loaded {cached_count} Form 4s from cache")
        if downloaded_count > 0:
            print(f"[{ticker}] Downloaded {downloaded_count} Form 4s")
        
        if insider_args:
            # Split into batches of 10 filings each
            batch_size = 10
            batches = [insider_args[i:i + batch_size] for i in range(0, len(insider_args), batch_size)]
            
            print(f"[{ticker}] Processing {len(batches)} batches with up to {batch_size} filings each")
            
            # Process batches in parallel
            with ProcessPoolExecutor(max_workers=8) as insider_executor:
                futures = {
                    insider_executor.submit(process_insider_batch, batch): batch 
                    for batch in batches
                }
                
                # Collect results with progress bar
                for future in tqdm(
                    as_completed(futures), 
                    total=len(futures),
                    desc=f"[{ticker}] Insider batches",
                    unit="batch"
                ):
                    transactions = future.result()
                    if transactions:
                        companies[ticker].insider_trades.extend(transactions)
            
            print(f"[{ticker}] ✓ Processed {len(companies[ticker].insider_trades)} insider transactions\n")
        
    except Exception as e:
        print(f"[{ticker}] ❌ Error processing company: {e}")
        import traceback
        traceback.print_exc()

print("\n" + "="*60)
print("EXTRACTION COMPLETE")
print("="*60)
print(f"Total companies processed: {len(companies)}")
print(f"Ready to ingest into Neo4j")

Processing companies:   0%|          | 0/20 [00:00<?, ?it/s]

[AAPL] Loading 10-K from cache...
[AAPL] Loading DEF 14A from cache...
[AAPL] Period of report: 2025-09-27
[AAPL] Filing Company: Apple Inc.
  [DEBUG] Skipping abstract concept: us-gaap_OperatingExpensesAbstract - 'Operating expenses:'
  [DEBUG] Skipping abstract concept: us-gaap_EarningsPerShareAbstract - 'Earnings per share:'
  [DEBUG] Skipping abstract concept: us-gaap_WeightedAverageNumberOfSharesOutstandingAbstract - 'Shares used in computing earnings per share:'


us-gaap_RevenueFromContractWithCustomerExcludingAssessedTaxLabel: Net sales
  ✓ MAPPED: us-gaap_RevenueFromContractWithCustomerExcludingAssessedTax → revenue = $416,161,000,000.00 (14 segments)
us-gaap_CostOfGoodsAndServicesSoldLabel: Cost of sales
  ✓ MAPPED: us-gaap_CostOfGoodsAndServicesSold → cost_of_sales = $220,960,000,000.00 (7 segments)
us-gaap_GrossProfitLabel: Gross margin
  ✓ MAPPED [PRIORITY]: us-gaap_GrossProfit → gross_profit = $195,201,000,000.00
us-gaap_ResearchAndDevelopmentExpenseLabel: Research and development
  ⊘ SKIPPED (unmapped): us-gaap_ResearchAndDevelopmentExpense - 'Research and development'
us-gaap_SellingGeneralAndAdministrativeExpenseLabel: Selling, general and administrative
  ⊘ SKIPPED (unmapped): us-gaap_SellingGeneralAndAdministrativeExpense - 'Selling, general and administrative'
us-gaap_OperatingExpensesLabel: Total operating expenses
  ✓ MAPPED [PRIORITY]: us-gaap_OperatingExpenses → operating_expenses = $62,151,000,000.00
us-gaap_OperatingIncomeLos

Truncated SGML: <DOCUMENT> at offset 1249741 has no matching </DOCUMENT>
No XBRL attachments found in filing Filing(company='Apple Inc.', cik=320193, form='DEF 14A', filing_date='2026-01-08', accession_no='0001308179-26-000008')


[AAPL] Found 20 insider filings
[AAPL] Loaded 20 Form 4s from cache
[AAPL] Processing 2 batches with up to 10 filings each


...

us-gaap_RevenuesLabel: Net revenues
  ✓ MAPPED [PRIORITY]: us-gaap_Revenues → revenue = $33,172,000,000.00 (7 segments)
pypl_TransactionExpenseLabel: Transaction expense
  ⊘ SKIPPED (unmapped): pypl_TransactionExpense - 'Transaction expense'
pypl_TransactionAndCreditLossesLabel: Transaction and credit losses
  ⊘ SKIPPED (unmapped): pypl_TransactionAndCreditLosses - 'Transaction and credit losses'
pypl_CustomerSupportAndOperationsExpenseLabel: Customer support and operations
  ⊘ SKIPPED (unmapped): pypl_CustomerSupportAndOperationsExpense - 'Customer support and operations'
us-gaap_SellingAndMarketingExpenseLabel: Sales and marketing
  ⊘ SKIPPED (unmapped): us-gaap_SellingAndMarketingExpense - 'Sales and marketing'
pypl_TechnologyAndDevelopmentExpenseLabel: Technology and development
  ⊘ SKIPPED (unmapped): pypl_TechnologyAndDevelopmentExpense - 'Technology and development'
us-gaap_GeneralAndAdministrativeExpenseLabel: General and administrative
  ⊘ SKIPPED (unmapped): us-gaap_GeneralAn

[PYPL] Found 27 insider filings
[PYPL] Loaded 27 Form 4s from cache
[PYPL] Processing 3 batches with up to 10 filings each


   🔴 SELL: 3962 acciones @ $59.80
   🔴 SELL: 200 acciones @ $60.57
   🔴 SELL: 7198 acciones @ $66.29
   🔴 SELL: 934 acciones @ $66.29
   🔴 SELL: 1150 acciones @ $66.30
   🔴 SELL: 10000 acciones @ $68.85
   🔴 SELL: 2500 acciones @ $68.96
   🔴 SELL: 1897 acciones @ $68.41
   🔴 SELL: 1800 acciones @ $69.47
   🔴 SELL: 141 acciones @ $70.20
   🔴 SELL: 5000 acciones @ $69.52
   🔴 SELL: 1374 acciones @ $69.13


Processing companies: 100%|██████████| 20/20 [05:39<00:00, 16.97s/it]

[PYPL] ✓ Processed 62 insider transactions


EXTRACTION COMPLETE
Total companies processed: 17
Ready to ingest into Neo4j


In [ ]:

import pickle
from pathlib import Path

PKL_PATH = Path("./companies_backup.pkl")


print(f"Backup file not found at {PKL_PATH.resolve()}. Creating new backup...")
PKL_PATH.parent.mkdir(parents=True, exist_ok=True)
with PKL_PATH.open("wb") as f:
    pickle.dump(companies, f)
print(f"Saved {len(companies)} companies to pickle file")


Backup file not found at /home/luis/Desktop/code/RAG/notebooks/companies_backup.pkl. Creating new backup...
Saved 17 funds to pickle file


In [ ]:

import pickle
from pathlib import Path

PKL_PATH = Path("./companies_backup.pkl")


PKL_PATH.parent.mkdir(parents=True, exist_ok=True)
with PKL_PATH.open("wb") as f:
    companies = pickle.load(companies, f)
print(f"Saved {len(companies)} funds to pickle file")



In [21]:
for key, company in companies.items():
    if company.filings_10k is not None:

        latest_date = max(company.filings_10k.keys())
        tenk = company.filings_10k[latest_date]

        print(tenk.filing_metadata)
        print(len(tenk.business_information))
        print(tenk.business_information)
        print(len(tenk.management_discussion_and_analysis))
        print(len(tenk.legal_proceedings))
        print(len(tenk.properties))
        print(len(tenk.income_statement_text))

        break

accession_number='0000320193-25-000079' filing_type='10-K' filing_date=datetime.date(2025, 10, 31) report_period_end=datetime.date(2025, 9, 27) filing_url='https://www.sec.gov/Archives/edgar/data/320193/0000320193-25-000079-index.html' cik='320193'
16054
Item 1.    Business

Company Background

The Company designs, manufactures and markets smartphones, personal computers, tablets, wearables and accessories, and sells a variety of related services. The Company’s fiscal year is the 52- or 53-week period that ends on the last Saturday of September.

Products

iPhone

iPhone® is the Company’s line of smartphones based on its iOS operating system. The iPhone line includes iPhone 17 Pro, iPhone Air™, iPhone 17, iPhone 16 and iPhone 16e.

Mac

Mac® is the Company’s line of personal computers based on its macOS® operating system. The Mac line includes laptops MacBook Air® and MacBook Pro®, as well as desktops iMac®, Mac mini®, Mac Studio® and Mac Pro®.

iPad

iPad® is the Company’s line of mul

### Generate Embeddings for Business Information

In [40]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Document
from tqdm import tqdm
from src.simple_rag.models.company import CompanyContentChunk
from simple_rag.embeddings.embedding import EmbedData
import re

# --- CONFIGURATION ---
# 10-K sections are dense and formal - balance context vs precision
FILING_CHUNK_SIZE = 512  
FILING_OVERLAP = 100
BATCH_SIZE = 32

print("⏳ Initializing 10-K Splitter...")
filing_splitter = SentenceSplitter(
    chunk_size=FILING_CHUNK_SIZE, 
    chunk_overlap=FILING_OVERLAP
)

embedder = EmbedData(
    model_name="nomic-ai/nomic-embed-text-v1.5", 
    batch_size=BATCH_SIZE
)

# Global queue for embedding
filing_tasks = []

# --- HELPER: EXTRACT SECTION HEADERS ---
def extract_section_info(text):
    """
    Detects section headers like 'Item 1. Business', 'Products', 'iPhone', etc.
    Returns (section_type, subsection) for metadata enrichment
    """
    # Match patterns like "Item 1.    Business" or standalone headers
    item_match = re.match(r'^(Item\s+\d+\.?\s*[A-Za-z\s]+)', text[:100], re.IGNORECASE)
    if item_match:
        return item_match.group(1).strip(), None
    
    # Match product/service category headers (usually title-cased, short)
    header_match = re.match(r'^([A-Z][a-zA-Z\s,&]+)\n', text[:50])
    if header_match and len(header_match.group(1).split()) <= 5:
        return None, header_match.group(1).strip()
    
    return None, None

print("🧠 Processing SEC 10-K Filings...")

# Process each company
for ticker, company in tqdm(companies.items(), desc="Chunking 10-K"):
    # Get the latest filing
    latest_filing = company.get_latest_filing()
    
    if not latest_filing:
        print(f"[{ticker}] No 10-K filing found, skipping...")
        continue
    
    # Process each section
    sections_to_process = [
        ("business_information", latest_filing.business_information, "business_information_chunks"),
        ("risk_factors", latest_filing.risk_factors, "risk_factors_chunks"),
        ("management_discussion_and_analysis", latest_filing.management_discussion_and_analysis, "management_discussion_chunks"),
        ("legal_proceedings", latest_filing.legal_proceedings, "legal_proceedings_chunks"),
        ("properties", latest_filing.properties, "properties_chunks"),
    ]
    
    total_chunks_for_company = 0
    
    for section_name, section_text, chunk_field_name in sections_to_process:
        # Skip empty sections
        if not section_text or section_text in ["N/A", "None", ""]:
            continue
            
        # 1. Create Document with metadata
        doc = Document(
            text=section_text,
            metadata={
                "cik": latest_filing.filing_metadata.cik,
                "filing_type": latest_filing.filing_metadata.filing_type,
                "filing_date": str(latest_filing.filing_metadata.filing_date),
                "report_period": str(latest_filing.filing_metadata.report_period_end),
                "section_name": section_name
            }
        )
        
        # 2. Split into chunks
        nodes = filing_splitter.get_nodes_from_documents([doc])
        
        # Reset chunk list for this section
        section_chunks = []
        
        for i, node in enumerate(nodes):
            # 3. Extract section context for better titles
            section_type, subsection = extract_section_info(node.text)
            
            # Build intelligent title
            if section_type:
                title = section_type
            elif subsection:
                title = f"{subsection} - Part {i+1}"
            else:
                title = f"{section_name.replace('_', ' ').title()} - Part {i+1}"
            
            # 4. Create CompanyContentChunk with full metadata
            new_chunk = CompanyContentChunk(
                id=f"{ticker}_{latest_filing.filing_metadata.filing_date}_{section_name}_chunk_{i}",
                title=title,
                text=node.text,
                embedding=[],  # Will be filled during embedding
                section_type=section_type,
                subsection=subsection,
                chunk_index=i,
                filing_cik=latest_filing.filing_metadata.cik,
                filing_date=str(latest_filing.filing_metadata.filing_date),
                section_name=section_name
            )
            
            # Add to section chunks and task queue
            section_chunks.append(new_chunk)
            filing_tasks.append((latest_filing, new_chunk, section_name))
        
        # Store chunks in the appropriate field on the filing object
        setattr(latest_filing, chunk_field_name, section_chunks)
        total_chunks_for_company += len(section_chunks)
        print(f"[{ticker}] {section_name}: {len(section_chunks)} chunks")
    
    print(f"[{ticker}] ✓ Total chunks: {total_chunks_for_company}\n")

# ==================================================
# BATCH EMBEDDING
# ==================================================
if filing_tasks:
    print(f"\n🔄 Embedding {len(filing_tasks)} 10-K chunks...")
    
    # Extract text (optionally prepend metadata for better embeddings)
    texts = []
    for filing, chunk, section_name in filing_tasks:
        # Enrich with context prefix for better semantic representation
        prefix = f"SEC Filing {chunk.filing_date}: "
        if chunk.section_type:
            prefix += chunk.section_type + " - "
        texts.append(prefix + chunk.text)
    
    # Embed
    vectors = embedder.embed(texts)
    
    # Assign embeddings back to chunks
    for i, vector in enumerate(vectors):
        filing_tasks[i][1].embedding = vector
        
    print(f"✅ 10-K processing complete. Generated {len(filing_tasks)} embedded chunks.")
else:
    print("⚠️ No filing content found.")

⏳ Initializing 10-K Splitter...


Cancellation requested; stopping current tasks.


KeyboardInterrupt: 

In [37]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Document
from src.simple_rag.models.company import CompanyContentChunk
from simple_rag.embeddings.embedding import EmbedData
import re
from collections import defaultdict

# --- CONFIGURATION ---
FILING_CHUNK_SIZE = 512  
FILING_OVERLAP = 100
BATCH_SIZE = 32

# --- DEBUG TRACKING ---
debug_stats = {
    "total_chunks": 0,
    "chunks_by_section": defaultdict(int),
    "chunk_sizes": [],
    "all_chunks": []
}

# --- HELPER: EXTRACT SECTION HEADERS ---
def extract_section_info(text):
    """
    Detects section headers like 'Item 1. Business', 'Products', 'iPhone', etc.
    Returns (section_type, subsection) for metadata enrichment
    """
    # Match patterns like "Item 1.    Business" or standalone headers
    item_match = re.match(r'^(Item\s+\d+\.?\s*[A-Za-z\s]+)', text[:100], re.IGNORECASE)
    if item_match:
        return item_match.group(1).strip(), None
    
    # Match product/service category headers (usually title-cased, short)
    header_match = re.match(r'^([A-Z][a-zA-Z\s,&]+)\n', text[:50])
    if header_match and len(header_match.group(1).split()) <= 5:
        return None, header_match.group(1).strip()
    
    return None, None

def debug_print_chunk(chunk, index):
    """Pretty print chunk details for debugging"""
    print(f"\n{'='*80}")
    print(f"📄 CHUNK #{index}")
    print(f"{'='*80}")
    print(f"🏷️  Title: {chunk.title}")
    print(f"🆔 ID: {chunk.id}")
    print(f"📏 Length: {len(chunk.text)} chars (~{len(chunk.text.split())} words)")
    print(f"🏢 Section Type: {chunk.section_type or 'None'}")
    print(f"📑 Subsection: {chunk.subsection or 'None'}")
    print(f"📊 Chunk Index: {chunk.chunk_index}")
    print(f"📂 Section Name: {chunk.section_name}")
    print(f"\n📝 Text Preview (first 300 chars):")
    print(f"{'-'*80}")
    print(chunk.text[:300] + "..." if len(chunk.text) > 300 else chunk.text)
    print(f"{'-'*80}")
    if len(chunk.text) > 300:
        print(f"\n📝 Text Preview (last 200 chars):")
        print(f"{'-'*80}")
        print("..." + chunk.text[-200:])
        print(f"{'-'*80}")

def process_and_debug_filing(text, cik="SAMPLE", filing_date="2025-01-01", section_name="business_information"):
    """
    Process a single filing text and show detailed debugging info
    
    Args:
        text: The filing text to chunk
        cik: Company CIK (for metadata)
        filing_date: Filing date (for metadata)
    """
    
    print("⏳ Initializing 10-K Splitter...")
    filing_splitter = SentenceSplitter(
        chunk_size=FILING_CHUNK_SIZE, 
        chunk_overlap=FILING_OVERLAP
    )
    
    print(f"\n{'#'*80}")
    print(f"🏦 Processing Filing: {cik} | Date: {filing_date}")
    print(f"📄 Total Text Length: {len(text)} chars (~{len(text.split())} words)")
    print(f"{'#'*80}")
    
    # 1. Create Document
    doc = Document(
        text=text,
        metadata={
            "cik": cik,
            "filing_type": "10-K",
            "filing_date": filing_date
        }
    )
    
    # 2. Split into chunks
    print(f"\n✂️  Splitting into chunks...")
    nodes = filing_splitter.get_nodes_from_documents([doc])
    print(f"✅ Split into {len(nodes)} chunks\n")
    
    content_chunks = []
    
    # 3. Process each chunk
    for i, node in enumerate(nodes):
        # Extract section context
        section, subsection = extract_section_info(node.text)
        
        # Build intelligent title
        if section:
            title = section
        elif subsection:
            title = f"{subsection} - Part {i+1}"
        else:
            title = f"Section {i+1}"
        
        # Create Chunk
        # Create CompanyContentChunk with metadata
        new_chunk = CompanyContentChunk(
            id=f"{cik}_{filing_date}_chunk_{i}",
            title=title,
            text=node.text,
            embedding=[],
            section_type=section,
            subsection=subsection,
            chunk_index=i,
            filing_cik=cik,
            filing_date=filing_date,
            section_name="business_information"  # or pass this as a parameter
        )
        
        # Track stats
        debug_stats["total_chunks"] += 1
        debug_stats["chunk_sizes"].append(len(node.text))
        section_key = section or subsection or "Unclassified"
        debug_stats["chunks_by_section"][section_key] += 1
        debug_stats["all_chunks"].append(new_chunk)
        
        content_chunks.append(new_chunk)
        
        # Print each chunk
        debug_print_chunk(new_chunk, i)
    
    # ==================================================
    # SUMMARY STATISTICS
    # ==================================================
    print(f"\n\n{'*'*80}")
    print(f"📊 CHUNKING STATISTICS")
    print(f"{'*'*80}")
    print(f"Total Chunks Generated: {debug_stats['total_chunks']}")
    print(f"Average Chunk Size: {sum(debug_stats['chunk_sizes']) / len(debug_stats['chunk_sizes']):.0f} chars")
    print(f"Min Chunk Size: {min(debug_stats['chunk_sizes'])} chars")
    print(f"Max Chunk Size: {max(debug_stats['chunk_sizes'])} chars")
    
    print(f"\n📑 Chunks by Section Type:")
    for section, count in sorted(debug_stats["chunks_by_section"].items(), key=lambda x: -x[1]):
        percentage = (count / debug_stats['total_chunks']) * 100
        print(f"   {section}: {count} chunks ({percentage:.1f}%)")
    
    print(f"\n📈 Chunk Size Distribution:")
    sizes = debug_stats['chunk_sizes']
    quartiles = [0, 25, 50, 75, 100]
    for q in quartiles:
        idx = int(len(sizes) * q / 100)
        if idx < len(sizes):
            print(f"   {q}th percentile: {sorted(sizes)[idx]} chars")
    
    return content_chunks


latest_date = max(companies["AAPL"].filings_10k.keys())
print(companies["AAPL"])
chunks = process_and_debug_filing(
    text=companies["AAPL"].filings_10k[latest_date].business_information,
    cik="320193",
    filing_date="2025-10-31",
    section_name="business_information"
)
   
    
print(f"\n✅ Processing complete! Generated {len(chunks)} chunks.")

name='Apple Inc.' ticker='AAPL' cik='320193' filings_10k={datetime.date(2025, 10, 31): Filing10K(filing_metadata=FilingMetadata(accession_number='0000320193-25-000079', filing_type='10-K', filing_date=datetime.date(2025, 10, 31), report_period_end=datetime.date(2025, 9, 27), filing_url='https://www.sec.gov/Archives/edgar/data/320193/0000320193-25-000079-index.html', cik='320193'), business_information='Item 1.\xa0\xa0\xa0\xa0Business\n\nCompany Background\n\nThe Company designs, manufactures and markets smartphones, personal computers, tablets, wearables and accessories, and sells a variety of related services. The Company’s fiscal year is the 52- or 53-week period that ends on the last Saturday of September.\n\nProducts\n\niPhone\n\niPhone® is the Company’s line of smartphones based on its iOS operating system. The iPhone line includes iPhone 17 Pro, iPhone Air™, iPhone 17, iPhone 16 and iPhone 16e.\n\nMac\n\nMac® is the Company’s line of personal computers based on its macOS® operati

In [20]:
for fund in funds_total:
    if hasattr(fund, 'derivatives') and fund.derivatives is not None:
        print(fund.derivatives)

date='2025-09-30' derivatives_df=  name ticker cusip  isin  lei shares market_value weight_pct currency  \
0  N/A   FAZ5   N/A  None  N/A   None         None       None     None   
1  N/A   NQZ5   N/A  None  N/A   None         None       None     None   
2  N/A  RTYZ5   N/A  None  N/A   None         None       None     None   
3  N/A   None   N/A  None  N/A   None         None       None     None   
4  N/A   None   N/A  None  N/A   None         None       None     None   
5  N/A   None   N/A  None  N/A   None         None       None     None   

  asset_category asset_category_desc issuer_category issuer_category_desc  \
0             DE                None           OTHER                 None   
1             DE                None           OTHER                 None   
2             DE                None           OTHER                 None   
3             DE                None           OTHER                 None   
4             DE                None           OTHER           

### Financial Highlights

In [16]:
years = set()
for fun in funds_total:
    if fun.financial_highlights is not None:
        years.update(fun.financial_highlights.keys())

print(years)

{'2015', '2019', '2026', '2034', '2000', '2016', '2027', '2031', '2017', '2018', '2044', '2023', '2022', '2029', '2028', '2020', '2054', '2024', '2025', '2021', '2030'}


In [17]:

count = 0
for fund in funds_total:
    if fund.financial_highlights is None:
        print(f"No financial highlights for {fund.ticker}")
        continue
    for year, highlight in fund.financial_highlights.items():
        neo.add_financial_highlight(
            fund_ticker=fund.ticker,
            year=year,
            turnover=highlight.turnover,
            expense_ratio=highlight.expense_ratio,
            total_return=highlight.total_return,
            net_assets=highlight.net_assets,
            net_assets_value_begining=highlight.net_assets_value_begining,
            net_assets_value_end=highlight.net_assets_value_end,
            net_income_ratio=highlight.net_income_ratio
        )
        count += 1
print(count)

✅ Added financial highlights for VEXMX (2024)
✅ Added financial highlights for VEXMX (2023)
✅ Added financial highlights for VEXMX (2022)
✅ Added financial highlights for VEXMX (2021)
✅ Added financial highlights for VEXMX (2020)
✅ Added financial highlights for VXF (2024)
✅ Added financial highlights for VXF (2023)
✅ Added financial highlights for VXF (2022)
✅ Added financial highlights for VXF (2021)
✅ Added financial highlights for VXF (2020)
✅ Added financial highlights for VEXAX (2024)
✅ Added financial highlights for VEXAX (2023)
✅ Added financial highlights for VEXAX (2022)
✅ Added financial highlights for VEXAX (2021)
✅ Added financial highlights for VEXAX (2020)
✅ Added financial highlights for VIEIX (2024)
✅ Added financial highlights for VIEIX (2023)
✅ Added financial highlights for VIEIX (2022)
✅ Added financial highlights for VIEIX (2021)
✅ Added financial highlights for VIEIX (2020)
✅ Added financial highlights for VEMPX (2024)
✅ Added financial highlights for VEMPX (2023

✅ Added financial highlights for VIGRX (2023)
✅ Added financial highlights for VIGRX (2022)
✅ Added financial highlights for VIGRX (2021)
✅ Added financial highlights for VIGRX (2020)
✅ Added financial highlights for VIGRX (2015)
✅ Added financial highlights for VIGRX (2016)
✅ Added financial highlights for VIGRX (2017)
✅ Added financial highlights for VIGRX (2018)
✅ Added financial highlights for VIGRX (2019)
✅ Added financial highlights for VUG (2024)
✅ Added financial highlights for VUG (2023)
✅ Added financial highlights for VUG (2022)
✅ Added financial highlights for VUG (2021)
✅ Added financial highlights for VUG (2020)
✅ Added financial highlights for VUG (2015)
✅ Added financial highlights for VUG (2016)
✅ Added financial highlights for VUG (2017)
✅ Added financial highlights for VUG (2018)
✅ Added financial highlights for VUG (2019)
✅ Added financial highlights for VIGAX (2024)
✅ Added financial highlights for VIGAX (2023)
✅ Added financial highlights for VIGAX (2022)
✅ Added 

### Sector allocation

In [ ]:
sectors = set()
move_to_geographies = [
    'North America', 
    'South America', 
    'Europe', 
    'Asia', 
    'Oceania', 
    'Africa'
]
for fund in funds_total:
    if fund.sector_allocation is not None:
        for value in move_to_geographies:
            if value in fund.sector_allocation.iloc[:, 0].tolist():
                if fund.geographic_allocation is None:
                    print("Updating Fund: ", fund.name)
                    fund.geographic_allocation = fund.sector_allocation
                    fund.sector_allocation = None
                    break
        sectors.update(fund.sector_allocation.iloc[:, 0].tolist())

print(sectors)



Updating Fund:  Vanguard ESG International Stock ETF
Updating Fund:  Vanguard Global Wellington™ Fund
Updating Fund:  Vanguard Global Wellington™ Fund
Updating Fund:  Vanguard Global Wellesley® Income Fund
Updating Fund:  Vanguard Global Wellesley® Income Fund
Updating Fund:  Vanguard International Growth Fund
Updating Fund:  Vanguard International Growth Fund
Updating Fund:  Vanguard Energy Fund
Updating Fund:  Vanguard Energy Fund
Updating Fund:  Vanguard Health Care Fund
Updating Fund:  Vanguard Health Care Fund
Updating Fund:  Vanguard Global Capital Cycles Fund
Updating Fund:  Vanguard Global ESG Select Stock Fund
Updating Fund:  Vanguard Global ESG Select Stock Fund
Updating Fund:  Vanguard Emerging Markets Government Bond Index Fund
Updating Fund:  Vanguard Emerging Markets Government Bond Index Fund
Updating Fund:  Vanguard Emerging Markets Government Bond Index Fund
Updating Fund:  Vanguard Global Minimum Volatility Fund
Updating Fund:  Vanguard Global Minimum Volatility Fund


In [18]:
sectors = 0
for fund in funds_total:
    if fund.sector_allocation is None:
        continue
    neo.add_sector_allocations(fund.sector_allocation, fund.ticker, fund.report_date) 
    sectors += 1
    
print(f"✅ Added {sectors} sector allocations")

✅ Added 12 sector allocations to VEXMX (2024-12-31)
✅ Added 12 sector allocations to VXF (2024-12-31)
✅ Added 12 sector allocations to VEXAX (2024-12-31)
✅ Added 12 sector allocations to VIEIX (2024-12-31)
✅ Added 12 sector allocations to VEMPX (2024-12-31)
✅ Added 12 sector allocations to VSEMX (2024-12-31)
✅ Added 12 sector allocations to VIMSX (2024-12-31)
✅ Added 12 sector allocations to VO (2024-12-31)
✅ Added 12 sector allocations to VIMAX (2024-12-31)
✅ Added 12 sector allocations to VMCIX (2024-12-31)
✅ Added 12 sector allocations to VMCPX (2024-12-31)
✅ Added 12 sector allocations to VMGIX (2024-12-31)
✅ Added 12 sector allocations to VOT (2024-12-31)
✅ Added 12 sector allocations to VMGMX (2024-12-31)
✅ Added 11 sector allocations to VMVIX (2024-12-31)
✅ Added 11 sector allocations to VOE (2024-12-31)
✅ Added 11 sector allocations to VMVAX (2024-12-31)
✅ Added 12 sector allocations to NAESX (2024-12-31)
✅ Added 12 sector allocations to VB (2024-12-31)
✅ Added 12 sector alloca

### Trailing Performance

In [19]:
%reload_ext autoreload
from src.simple_rag.database.neo4j.neo4j import Neo4jDatabase
neo = Neo4jDatabase()
count = 0
for fund in funds_total:
    if fund.performance is not None:
        for date_key, perf_snapshot in fund.performance.items():
            neo.create_performance_node(fund.ticker, perf_snapshot, date_key)
            count += 1
print(count)

python-dotenv could not parse statement starting at line 3


[{'tp': {'return5y': 9.75, 'return10y': 9.31, 'id': 'VEXMX_2024', 'return1y': 16.76}}]
[{'tp': {'return5y': 9.89, 'return10y': 9.45, 'id': 'VXF_2024', 'return1y': 16.9}}]
[{'tp': {'return5y': 9.89, 'return10y': 9.45, 'id': 'VEXAX_2024', 'return1y': 16.91}}]
[{'tp': {'return5y': 9.9, 'return10y': 9.47, 'id': 'VIEIX_2024', 'return1y': 16.91}}]
[{'tp': {'return5y': 9.91, 'return10y': 9.48, 'id': 'VEMPX_2024', 'return1y': 16.94}}]
[{'tp': {'return5y': 9.94, 'id': 'VSEMX_2024', 'return1y': 16.96, 'returnInception': 12.09}}]
[{'tp': {'return5y': 9.72, 'return10y': 9.42, 'id': 'VIMSX_2024', 'return1y': 15.09}}]
[{'tp': {'return5y': 9.85, 'return10y': 9.56, 'id': 'VO_2024', 'return1y': 15.23}}]
[{'tp': {'return5y': 9.85, 'return10y': 9.55, 'id': 'VIMAX_2024', 'return1y': 15.22}}]
[{'tp': {'return5y': 9.86, 'return10y': 9.56, 'id': 'VMCIX_2024', 'return1y': 15.23}}]
[{'tp': {'return5y': 9.87, 'return10y': 9.57, 'id': 'VMCPX_2024', 'return1y': 15.25}}]
[{'tp': {'return5y': 10.44, 'return10y': 10

### Add Geographic Allocation

In [23]:
geographics = set()
for fund in funds_total:
    if fund.geographic_allocation is not None:
        geographics.update(fund.geographic_allocation.iloc[:, 0].tolist())

print(geographics)

{'France', 'Supranational', 'Australia', 'Israel', 'Finland', 'Mexico', 'Saudi Arabia', 'Taiwan', 'Spain', 'Philippines', 'India', 'United Arab Emirates', 'Colombia', 'Sweden', 'Denmark', 'Japan', 'Norway', 'Belgium', 'South Korea', 'New Zealand', 'United Kingdom', 'China', 'Chile', 'Switzerland', 'Turkey', 'Portugal', 'Qatar', 'Poland', 'Thailand', 'Indonesia', 'Other', 'South Africa', 'Singapore', 'Brazil', 'Canada', 'Malaysia', 'Italy', 'Hong Kong', 'Germany', 'United States', 'Peru', 'Netherlands'}


In [20]:
geographic = 0
for fund in funds_total:
    if fund.geographic_allocation is None:
        continue
    neo.add_geographic_allocations(fund.geographic_allocation, fund.ticker, fund.report_date) 
    geographic += 1
    
print(f"✅ Added {geographic} geographic allocations")

✅ Added 10 geographic allocations to ERET (2025-04-30)
⚠️ No geographic data provided for ICLN
✅ Added 10 geographic allocations to REET (2025-04-30)
✅ Added 10 geographic allocations to IFGL (2025-04-30)
⚠️ No geographic data provided for IDV
✅ Added 11 geographic allocations to DMXF (2025-08-31)
✅ Added 11 geographic allocations to EMXF (2025-08-31)
✅ Added 11 geographic allocations to ESGD (2025-08-31)
✅ Added 11 geographic allocations to LDEM (2025-08-31)
✅ Added 11 geographic allocations to EQLT (2025-08-31)
✅ Added 11 geographic allocations to EVLU (2025-08-31)
✅ Added 11 geographic allocations to AQLT (2025-08-31)
✅ Added 11 geographic allocations to SDG (2025-08-31)
✅ Added 5 geographic allocations to AIA (2025-03-31)
✅ Added 7 geographic allocations to EMIF (2025-03-31)
⚠️ No geographic data provided for IEV
✅ Added 7 geographic allocations to ARTY (2025-03-31)
✅ Added 11 geographic allocations to IOO (2025-03-31)
⚠️ No geographic data provided for IXP
⚠️ No geographic data pr

### Add the Managers

In [21]:
for fund in tqdm(funds_total):
    if hasattr(fund, 'managers') and fund.managers is not None and len(fund.managers) > 0:
        neo.add_managers(fund.ticker, fund.managers)

  0%|          | 0/420 [00:00<?, ?it/s]

👥 Adding 3 managers to fund VEXMX
✅ Added manager: Walter Nejman to VEXMX


  0%|          | 1/420 [00:00<00:49,  8.40it/s]

✅ Added manager: Michelle Louie to VEXMX
✅ Added manager: Nick Birkett to VEXMX
👥 Adding 3 managers to fund VXF
✅ Added manager: Walter Nejman to VXF
✅ Added manager: Michelle Louie to VXF
✅ Added manager: Nick Birkett to VXF
👥 Adding 3 managers to fund VEXAX
✅ Added manager: Walter Nejman to VEXAX
✅ Added manager: Michelle Louie to VEXAX
✅ Added manager: Nick Birkett to VEXAX
👥 Adding 3 managers to fund VIEIX


  1%|          | 4/420 [00:00<00:23, 17.54it/s]

...

✅ Added manager: Matt Waldron to ACWX
👥 Adding 3 managers to fund CRBN
✅ Added manager: Jennifer Hsui to CRBN
✅ Added manager: Steven White to CRBN
✅ Added manager: Matt Waldron to CRBN
👥 Adding 3 managers to fund AAXJ
✅ Added manager: Jennifer Hsui to AAXJ
✅ Added manager: Steven White to AAXJ
✅ Added manager: Matt Waldron to AAXJ
👥 Adding 3 managers to fund CNYA
✅ Added manager: Jennifer Hsui to CNYA
✅ Added manager: Steven White to CNYA
✅ Added manager: Matt Waldron to CNYA
👥 Adding 3 managers to fund EFA
✅ Added manager: Jennifer Hsui to EFA
✅ Added manager: Steven White to EFA
✅ Added manager: Matt Waldron to EFA
👥 Adding 3 managers to fund EFG
✅ Added manager: Jennifer Hsui to EFG
✅ Added manager: Steven White to EFG
✅ Added manager: Matt Waldron to EFG
👥 Adding 3 managers to fund EFAV
✅ Added manager: Jennifer Hsui to EFAV
✅ Added manager: Steven White to EFAV
✅ Added manager: Matt Waldron to EFAV
👥 Adding 3 managers to fund SCZ
✅ Added manager: Jennifer Hsui to SCZ
✅ Added mana

 76%|███████▋  | 321/420 [00:07<00:02, 42.15it/s]

✅ Added manager: Steven White to TOK
✅ Added manager: Matt Waldron to TOK
👥 Adding 3 managers to fund IBRN
✅ Added manager: Jennifer Hsui to IBRN
✅ Added manager: Steven White to IBRN
✅ Added manager: Matt Waldron to IBRN
👥 Adding 3 managers to fund IDRV
✅ Added manager: Jennifer Hsui to IDRV
✅ Added manager: Steven White to IDRV
✅ Added manager: Matt Waldron to IDRV
👥 Adding 2 managers to fund IUSB
✅ Added manager: James Mauro to IUSB
✅ Added manager: Jonathan Graves to IUSB
👥 Adding 2 managers to fund EUSB


100%|██████████| 420/420 [00:07<00:00, 54.85it/s]

✅ Added manager: James Mauro to EUSB
✅ Added manager: Jonathan Graves to EUSB


## Text2Cypher Model

In [21]:
for fund in funds_total:
    if "Vanguard Total Stock Market" in fund.name and fund.non_derivatives is not None:
        print(fund.ticker)
    

VTSMX


In [1]:
 # Cell: Text-to-Cypher Translation Demo
from pathlib import Path
import sys

RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG/")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))
SRC_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG/src")
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))
%reload_ext autoreload

from simple_rag.rag.text2cypher import CypherTranslator
from simple_rag.database.neo4j.neo4j import Neo4jDatabase
import json
import time
from simple_rag.evaluation.benchmark import Text2CypherBenchmark


MODEL = "tomasonjo/llama3-text2cypher-demo:8b_4bit"
print("Current path: ", Path.cwd())
PATH = Path("../src/simple_rag/evaluation/test_set.json")
GROQ_MODEL = "llama-3.3-70b-versatile"
backend = "ollama"
if backend == "groq":
    benchmark = Text2CypherBenchmark(PATH, GROQ_MODEL, backend="groq")
    try:
        benchmark.run(complexity_filter=["hard"])
    except KeyboardInterrupt:
        print("\nBenchmark stopped by user.")
        benchmark.cleanup()
else:
    benchmark = Text2CypherBenchmark(PATH, MODEL, backend="ollama", interactive=False)
    try:
        benchmark.run()
    except KeyboardInterrupt:
        print("\nBenchmark stopped by user.")
        benchmark.cleanup()



python-dotenv could not parse statement starting at line 3
python-dotenv could not parse statement starting at line 3


✓ Loaded .env from: ../.env
Current path:  /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks


Skipping import of cpp extensions due to incompatible torch version 2.8.0+cu128 for torchao version 0.15.0             Please see https://github.com/pytorch/ao/issues/2919 for more info


✓ Cache valid (Hash: 9580404d...). Loading vector index...
Loading Entity Cache...
Cache Loaded: 348 Funds, 2 Providers
✓ EntityResolver initialized
Ollama server not detected, attempting to start...
Starting Ollama server...
✓ Ollama server started successfully (took 2s)
✓ Ollama LangChain initialized with model: tomasonjo/llama3-text2cypher-demo:8b_4bit
🔥 Warming up LLM (loading into memory)...
✓ LLM Warmed up and ready.

🚀 Text-to-Cypher Benchmark Suite
📊 Total Questions: 98
🤖 Model: tomasonjo/llama3-text2cypher-demo:8b_4bit
🔧 Backend: ollama


📝 Q1: What is the full name of the fund with ticker VTI?

Fund Search - Top 5 matches:
  1. 'iShares Future AI & Tech ETF' (ARTY) - Score: 58.9
  2. 'iShares Future Cloud 5G and Tech ETF' (IDAT) - Score: 58.1
  3. 'iShares GNMA Bond ETF' (GNMA) - Score: 57.6
  4. 'iShares Global Energy ETF' (IXC) - Score: 57.2
  5. 'iShares Russell Mid-Cap ETF' (IWR) - Score: 57.0

Top score 58.9 below threshold 60, no matches returned
Resolved Entities:  {'V